# Create a virtual reference to a year's worth of data

In [1]:
import earthaccess 
import xarray as xr
from virtualizarr import open_virtual_dataset
import dask

# To deal with NASA earthdata login :(
import netrc 
import fsspec
import aiohttp

In [2]:
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2024-01-01", "2024-05-31"))

Granules found: 122


In [3]:
data_links = [granule.data_links(access="external")[0] for granule in results] #access='direct' for in-region AWS S3 us-west-2

In [4]:
(username, account, password) = netrc.netrc().authenticators("urs.earthdata.nasa.gov")
fsspec.config.conf['https'] = dict(client_kwargs={'auth': aiohttp.BasicAuth(username, password)})

In [9]:
# Parallelize open_virtual_dataset
lazy_open_virtual_dataset = dask.delayed(open_virtual_dataset)
lazy_vds = [lazy_open_virtual_dataset(f, filetype='hdf5', indexes={},  loadable_variables='time', reader_options={}) for f in data_links]


In [10]:
%%time 
# this takes a while b/c we open and scan all 122 remote netCDF files
vds_list = dask.compute(*lazy_vds) 

CPU times: user 6.8 s, sys: 1.88 s, total: 8.68 s
Wall time: 1min 18s


In [11]:
combined_vds = xr.concat(vds_list, dim='time', coords='minimal', compat='override')
combined_vds.virtualize.to_kerchunk('M2SDNXSLV.json', format='json')

In [12]:
combined_vds.virtualize.to_kerchunk('M2SDNXSLV.json', format='json')

In [13]:
!ls -ltrh M2SDNXSLV.json

-rw-r--r--@ 1 scotthenderson  staff   1.5M Jun 14 17:30 M2SDNXSLV.json


In [14]:
# Now we can use this 1.5MB file for efficient future access
ds = xr.open_dataset('M2SDNXSLV.json', engine='kerchunk')
ds

<xarray.Dataset> Size: 507MB
Dimensions:     (time: 122, lat: 361, lon: 576)
Coordinates:
  * lat         (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon         (lon) float64 5kB -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * time        (time) datetime64[ns] 976B 2024-01-01T12:00:00 ... 2024-05-01...
Data variables:
    HOURNORAIN  (time, lat, lon) float32 101MB ...
    T2MMAX      (time, lat, lon) float32 101MB ...
    T2MMEAN     (time, lat, lon) float32 101MB ...
    T2MMIN      (time, lat, lon) float32 101MB ...
    TPRECMAX    (time, lat, lon) float32 101MB ...
Attributes: (12/31)
    Comment:                           GMAO filename: d5124_m2_jan10.statD_2d...
    Contact:                           http://gmao.gsfc.nasa.gov
    Conventions:                       CF-1
    DataResolution:                    0.5 x 0.625
    EasternmostLongitude:              179.375
    Filename:                          MERRA2_400.statD_2d_slv_Nx.20240101.nc4
    ...                                ...
    TemporalRange:                     1980-01-01 -> 2016-12-31
    Title:                             MERRA2 statD_2d_slv_Nx: 2d,Daily,Aggre...
    VersionID:                         5.12.4
    WesternmostLongitude:              -180.0
    identifier_product_doi:            10.5067/9SC1VNTWGWV3
    identifier_product_doi_authority:  http://dx.doi.org/